# Introduction to custom ops in FINN

Suppose that you want to introduce a new (custom) operation type into the FINN. Custom operations in FINN are useful for a variety of things ranging from code generation to functional verification. This is achieved by creating a new Python module for your custom operation that fulfills certain interface specifications.

One thing to point out before we start is that **these custom operations are generic** and not really tied to e.g. Vivado HLS or few-bit quantization. As you will see in this notebook, it's possible to provide arbitrary Python/C/C++/... execution and code generation paths for custom nodes.

## The CustomOp base class

Subclasses of `CustomOp` provide a way of providing custom functionality for ONNX nodes in FINN.
This is the base class for every custom op node used in the framework, so you must create subclasses of `CustomOp` to provide execution, code generation and other functionalities in FINN.

Let's start by looking at the `CustomOp` base class itself, which lives in the `finn-base` repository. You can view it [here](https://github.com/Xilinx/finn-base/blob/dev/src/finn/custom_op/base.py). Note that the `finn` Docker container already has `finn-base` set up as a dependency.

Some points of importance:

1. `CustomOp` instances (in Python) are not meant to store any data, only provide functionality on top of data stored in ONNX. Each `CustomOp` instance has a member `self.onnx_node` which gives access to the ONNX `NodeProto` with attributes. There is also a custom attribute setter/getter system in `CustomOp` to make this process easier.

2. `CustomOp` subclasses need to implement the methods below (those not starting with underscore).

3. To be discoverable in the custom op register, `CustomOp` subclasses must set the `domain` field to the name of the Python module they appear in. For instance, to use the custom `Im2Col` op type from [here](https://github.com/Xilinx/finn-base/blob/dev/src/finn/custom_op/general/im2col.py), the ONNX node must use `domain=qonnx.custom_op.general` since its module is located at `finn/custom_op/general/im2col.py`.

In [ ]:
from finn.custom_op.base import CustomOp
dir(CustomOp)

## A Simple CustomOp Example

Let's make a simple CustomOp that raises its input to a given exponent (specified as attribute). For now it'll only work in Python, but later we'll add C++ execution capability too.

In [ ]:
from onnx import helper
import numpy as np

class MyPythonPowerOp(CustomOp):
    
    # here we use the CustomOp attribute system to make it easier
    # to set/get custom attributes on this node
    def get_nodeattr_types(self):
        return {
            # each entry is:
            # name of attribute : (dtype, required, default value)
            # dtype follows the ONNX attribute protobuf so
            # "i" is int, "s" is string, "f" is float,
            # "ints" is a list of integers...
            # also good practice to document what each attribute does here:
            
            # which integer power to raise the input to
            "exponent" : ("i", True, 0),
            # execution mode : currently only python
            "exec_mode" : ("s", True, "python"),
        }
    
    # return an ONNX node that has the same shape inference behavior
    # here we want in shape = out shape, so we use the ONNX ReLU
    # node to mimic its shape inference behavior
    # we have access to the entire ModelWrapper to help make this decision
    # (the parameter called model)
    def make_shape_compatible_op(self, model):
        node = self.onnx_node
        # make a Relu node connected to the same in-out tensors to get
        # shape inference
        # a general-purpose alternative is to use a Constant node that 
        # produces the desired shape
        return helper.make_node("Relu", [node.input[0]], [node.output[0]])

    # used for FINN DataType inference: set the output tensors' datatypes
    # accordingly for this node
    # here we assume input datatype = output datatype
    # we have access to the entire ModelWrapper to help make this decision
    # (the parameter called model)
    def infer_node_datatype(self, model):
        node = self.onnx_node
        # data type stays the same
        dtype = model.get_tensor_datatype(node.input[0])
        model.set_tensor_datatype(node.output[0], dtype)
    
    # execute this node
    # context: used for both input and output, dictionary of named
    #          tensors
    # graph: the ONNX GraphProto (ModelWrapper.graph), generally 
    #         not needed to execute a single node
    def execute_node(self, context, graph):
        exec_mode = self.get_nodeattr("exec_mode")
        if exec_mode == "python":
            # get names of node input and output tensors
            i_name = self.onnx_node.input[0]
            o_name = self.onnx_node.output[0]
            # grab input tensor from context
            i_tensor = context[i_name]
            # get which power to raise to from attribute
            expnt = self.get_nodeattr("exponent")
            # compute and put output into context
            o_tensor = np.power(i_tensor, expnt)
            context[o_name] = o_tensor
        else:
            raise Exception("Only python exec_mode is supported")
        
    # can use to do a sanity check of all the node's properties
    # optional, not implemented here
    def verify_node(self):
        pass
        
        

To make sure our custom op is available, it needs to be registered. The best practice for this is to create a submodule under `finn.custom_op` which includes a `custom_op` dictionary that maps strings (op names) to classes (op implementations). Since we're in a Jupyter notebook we'll just hijack it at runtime like this:

In [ ]:
import qonnx.custom_op.general as general
general.custom_op["MyPythonPowerOp"] = MyPythonPowerOp

We can see which custom ops are registered under this submodule by looking at the dictionary:

In [ ]:
general.custom_op

## Let's Try Out our CustomOp

We'll manually build a small ONNX graph containing our node in order to try out some of the functionality. This would normally go into the unit test for this CustomOp. 

The graph is built by first specifying the input/output tensor information (name, type, shape). Then,the custom node is generated; which is later used to generate the graph along the input/output tensor information. The model is built using the graph.  Finally, the model is wrapped around using the ModelWrapper function from FINN.

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
from onnx import TensorProto

def make_graph(ishape, exp, op_type = "MyPythonPowerOp"):
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, ishape
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, ishape
    )

    custom_node = helper.make_node(
        # op type string in ONNX, what we used to register the custom op
        op_type,
        # name of input tensor
        ["inp"],
        # name of output tensor
        ["outp"],
        # specify domain s.t. FINN can find our op under this submodule
        domain="qonnx.custom_op.general",
        # set up attributes
        exponent = int(exp),
        exec_mode = "python"
    )

    graph = helper.make_graph(
        nodes=[custom_node], name="custom_graph", inputs=[inp], outputs=[outp]
    )
    model = helper.make_model(graph, producer_name="custom-model")
    return ModelWrapper(model)

Now, we specify the input tensor shape and we generate the graph using the function we have just created. The input tensor shape and the exponent value are passed as parameters. These parameters are used to generate our model, graph and custom node using the `MyPythonPowerOp` operation.

In [ ]:
# generate a small graph with our custom op
input_shape = (1, 2, 4)
ret_model = make_graph(input_shape, 2)
ret_model.model.graph.node

We generate a random tensor based on the `input_shape` defined before. See the shape and values of the `random_input` below and the datatype. 

In [ ]:
from qonnx.core.datatype import DataType
from qonnx.util.basic import gen_finn_dt_tensor

# generate a random input of e.g signed 4-bit values
random_input = gen_finn_dt_tensor(DataType["INT4"], input_shape)
random_input


Just generate an Input Dictionary with the random values just generated. Then we execute the model using the model and random values just generated. 

In [ ]:
from finn.core.onnx_exec import execute_onnx

# run with FINN's execute_onnx
inp_dict = {"inp" : random_input}
ret = execute_onnx(ret_model, inp_dict)
ret

Done! We have just executed the model that uses our custom operation. The result should be the input number to the power of 2.

## A CustomOp with C++ Generation

We can write our CustomOps in C++ for instance and generate a model the same way we have done it previously. This can be done through python bindings that let us call C++ code from python. In fact, we will compile the C++ code and execute it from python. 

The following class is based on the `MyPythonPowerOp` class previously written. We are adding a new attribute `codegen_dir` into the `get_nodeattr_types` function that specifies the directory for the generated C++ code, building script and executable application.

We define a new function that `my_custom_cpp_gen` that writes the C++ code into a file and builds it. Finally the `execute_node` function is modified to support the C++ execution of the CustomOp. The `c++` branch of the if-else statements first flattens the input tensor and writes it into the "input.txt" file. Then the C++ compiled application is executed using bash commands. The application reads the ".txt" file, calculates the power value based on the exponent, and writes the result back into the "output.txt" file. Then the result of the ouput file is read and reshaped back into the original shape. Finally, the result is written into the `context` dictionary

In [ ]:
from finn.util.basic import make_build_dir, CppBuilder
import subprocess

# derive from our previous example
class MyMixedPowerOp(MyPythonPowerOp):
    
    # here we use the CustomOp attribute system to make it easier
    # to set/get custom attributes on this node
    def get_nodeattr_types(self):
        return {
            # each entry is:
            # name of attribute : (dtype, required, default value)
            # dtype follows the ONNX attribute protobuf so
            # "i" is int, "s" is string, "f" is float,
            # "ints" is a list of integers...
            # also good practice to document what each attribute does here:
            
            # which integer power to raise the input to
            "exponent" : ("i", True, 0),
            # execution mode : python or c++
            "exec_mode" : ("s", True, "python"),
            # code generation directory
            "codegen_dir" : ("s", False, ""),
        }
    
    def my_custom_cpp_gen(self):
        codegen_dir = make_build_dir(prefix="my_custom_op")
        # set attribute for codegen dir
        self.set_nodeattr("codegen_dir", codegen_dir)
        # generate some C++ code
        cpp_code = """
#include <iostream>
#include <fstream>
using namespace std;
#define EXPONENT %d

int main(int argc, char **argv) {
    ifstream infile("input.txt");
    ofstream outfile("output.txt");
    
    float elem;
    while (infile >> elem)
    {
        float res = 1.0;
        for(int i=0; i < EXPONENT; i++) {
            res *= elem;
        }
        outfile << res << "\\n";
    }

    return 0;
}
        """ % (self.get_nodeattr("exponent"))
        with open(codegen_dir+"/top.cpp", "w") as f:
            f.write(cpp_code)
        builder = CppBuilder()
        # to enable additional debug features please uncommand the next line
        builder.append_includes("--std=c++11")
        builder.append_includes("-O3")
        builder.append_sources(codegen_dir + "/*.cpp")
        builder.set_executable_path(codegen_dir + "/node_model")
        builder.build(codegen_dir)
    
    # execute this node
    # context: used for both input and output, dictionary of named
    #          tensors
    # graph: the ONNX GraphProto (ModelWrapper.graph), generally 
    #         not needed to execute a single node
    def execute_node(self, context, graph):
        exec_mode = self.get_nodeattr("exec_mode")
        # get names of node input and output tensors
        i_name = self.onnx_node.input[0]
        o_name = self.onnx_node.output[0]
        # grab input tensor from context
        i_tensor = context[i_name]
        # get which power to raise to from attribute
        expnt = self.get_nodeattr("exponent")
        if exec_mode == "python":
            # compute and put output into context
            o_tensor = np.power(i_tensor, expnt)
            context[o_name] = o_tensor
        elif exec_mode == "c++":
            build_dir = self.get_nodeattr("codegen_dir")
            # save input as txt, could preprocess, change layout etc..
            np.savetxt(build_dir+"/input.txt", i_tensor.flatten())
            bash_command = ["./node_model"]
            proc_run = subprocess.Popen(bash_command, cwd=build_dir, stdout=subprocess.PIPE)
            proc_run.communicate()
            o_tensor = np.loadtxt(build_dir+"/output.txt")
            o_tensor = o_tensor.reshape(i_tensor.shape)
            context[o_name] = o_tensor
        else:
            raise Exception("Only python and c++ exec_mode is supported")
        
    # can use to do a sanity check of all the node's properties
    # optional, not implemented here
    def verify_node(self):
        pass
        
        

We just register the new CustomOp the same way as we did before. Then, we create another graph using the same function `make_graph` as before. We can see the node containing the custom operation printed below.

In [ ]:
# register our new op
general.custom_op["MyMixedPowerOp"] = MyMixedPowerOp

# make graph with new op
mixedop_graph = make_graph(input_shape, 2, op_type = "MyMixedPowerOp")
mixedop_graph.graph.node

We just print all the functions inside the CustomOp, the default C++ code directory and the `exec_mode` attribute.

In [ ]:
from qonnx.custom_op.registry import getCustomOp

# get FINN wrapper for this node, with all the functionality
op_inst = getCustomOp(mixedop_graph.model.graph.node[0])
print("Available functions: " + str(dir(op_inst)))
# query some attributes
print("codegen_dir: " + op_inst.get_nodeattr("codegen_dir"))
print("exec_mode: " + op_inst.get_nodeattr("exec_mode"))

## Implement a code generation transformation

We define a local transformation function that transforms a specific model by accessing and modifying the attributes of the specified node. It will execute the `my_custom_cpp_gen` function from the node "MyMixedPowerOp" if the "codegen_dir" is not present.

In [ ]:
#from qonnx.transformation.base import Transformation
# can derive from NodeLocalTransformation for faster (parallel) execution
from qonnx.transformation.base import NodeLocalTransformation
import os

class MyNodeLocalCodeGen(NodeLocalTransformation):
    
    # will get called (possibly in parallel) for each node
    def applyNodeLocal(self, node):
        # keep track whether we changed anything
        modified_graph = False
        # check node type before we do anything
        if node.op_type == "MyMixedPowerOp":
            # get FINN wrapper for this node, with all the functions
            op_inst = getCustomOp(node)
            if not os.path.isdir(op_inst.get_nodeattr("codegen_dir")):
                # call the codegen function we defined
                # this will modify the underlying node by setting attribute
                op_inst.my_custom_cpp_gen()
                # codegen function modifies attribute
                modified_graph = True
        # important: must return modified_graph = False at some point
        # otherwise transformation will run in infinite loop!
        return (node, modified_graph)

Apply the transformation into the model we had before. The returned model is the same input model after applying the specified transformation. 

In [ ]:
mixedop_graph_new = mixedop_graph.transform(MyNodeLocalCodeGen())

Print the "codegen_dir" attribute from CustomOp node.

In [ ]:
new_op_inst = getCustomOp(mixedop_graph_new.graph.node[0])
codegen_dir = new_op_inst.get_nodeattr("codegen_dir")
print(codegen_dir)

We can see that the `codegen_dir` folder contains the compile script, compiled application and the C++ source file:

In [ ]:
! ls {codegen_dir}

Let's view the content of the C++ source file:

In [ ]:
! cat {codegen_dir}/top.cpp

### Manually generate input and run C++ node model

We will now manually generate the input data and write it into the `input.txt` file. Then, we manually execute the compiled application and finally see the result in the `output.txt` file. 

The purpose of this is mostly to show that there is no "magic" happening when FINN is executing our custom op; it's just launching a program. When debugging the execution of your custom op, it's a good idea to keep this in mind -- for instance, you can use `gdb` to debug the internals of the C++ node model here.

In [ ]:
! echo "7.0 8.0 9.0" > {codegen_dir}/input.txt

In [ ]:
! cd {codegen_dir}; ./node_model

In [ ]:
! cat {codegen_dir}/output.txt

In [ ]:
! rm {codegen_dir}/*.txt

### Use FINN execution flow

We'll now trigger the custom node execution from inside FINN, via the custom ONNX execution capabilities which will automatically launch the appropriate handler when a custom node is encountered inside the ONNX graph, in this case launching the compiled C++ program. To do this, we will first generate a random tensor with a pre-specified tensor shape and print it. 

In [ ]:
# generate a random input of e.g signed 4-bit values
random_input = gen_finn_dt_tensor(DataType["INT4"], input_shape)
random_input

We set the CustomOp node attribute to execute in "Python" mode. Then, generate an input dictionay with the random input tensor and execute the transformed model using the `execute_onnx`. We print the output to see the results.

In [ ]:
# run with FINN's execute_onnx, custom node will use Python execution
new_op_inst.set_nodeattr("exec_mode", "python")
inp_dict = {"inp" : random_input}
ret = execute_onnx(mixedop_graph_new, inp_dict)
ret

We repeat the previous process in "c++" execution mode. 

In [ ]:
# run with FINN's execute_onnx, custom node will use c++ execution
new_op_inst.set_nodeattr("exec_mode", "c++")
ret = execute_onnx(mixedop_graph_new, inp_dict)
ret